# Learning Python (Day 52)

# Python Flask - User Roles

## **1. User Roles and Permissions**

**User Roles:** Roles represent different levels of access and permissions within the application. Common roles include:

- **Admin:** Full access to all functionalities.

- **Moderator:** Limited administrative capabilities.

- **User:** Basic access, typically for regular users.

**Permissions:** These define what actions each role can perform. For example:

- **READ:** Ability to view content.

- **WRITE:** Ability to create or modify content.

- **ADMIN:** Ability to manage users and settings.


## **2. Database Models for Roles and Permissions**

To manage roles and permissions, additional database models are necessary:

**Role Model:**
```python
class Role(db.Model):
    __tablename__ = 'roles'
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(64), unique=True)
    default = db.Column(db.Boolean, default=False, index=True)
    permissions = db.Column(db.Integer)
    users = db.relationship('User', backref='role', lazy='dynamic')

    def __repr__(self):
        return '<Role %r>' % self.name
```

**User Model:**
Modify the `User` model to include a reference to the role.
```python
class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id'))

    def __repr__(self):
        return '<User %r>' % self.username
```

## **3. Assigning Roles to Users**

When creating a new user, assign a default role. This can be done by extending the `User` model with additional methods.

**Assign Default Role:**
```python
class User(db.Model):
    # existing fields...
    role_id = db.Column(db.Integer, db.ForeignKey('roles.id'))

    def __init__(self, **kwargs):
        super(User, self).__init__(**kwargs)
        if self.role is None:
            if self.email == current_app.config['FLASKY_ADMIN']:
                self.role = Role.query.filter_by(permissions=0xff).first()
            if self.role is None:
                self.role = Role.query.filter_by(default=True).first()
```

## **4. Managing Role Permissions**

Define permissions using bitmasks, which allow combining multiple permissions using bitwise operations.

**Permission Constants:**
```python
class Permission:
    FOLLOW = 0x01
    COMMENT = 0x02
    WRITE_ARTICLES = 0x04
    MODERATE_COMMENTS = 0x08
    ADMINISTER = 0x80
```

Assign permissions to roles using these constants:
```python
class Role(db.Model):
    # existing fields...
    permissions = db.Column(db.Integer)

    def add_permission(self, perm):
        if not self.has_permission(perm):
            self.permissions += perm

    def remove_permission(self, perm):
        if self.has_permission(perm):
            self.permissions -= perm

    def reset_permissions(self):
        self.permissions = 0

    def has_permission(self, perm):
        return self.permissions & perm == perm
```

## **5. Role-Based Access Control (RBAC)**

Implement role-based access control to restrict certain views or actions to users with specific roles.

**Decorator for Permissions:**
```python
from functools import wraps
from flask import abort
from flask_login import current_user

def permission_required(permission):
    def decorator(f):
        @wraps(f)
        def decorated_function(*args, **kwargs):
            if not current_user.can(permission):
                abort(403)
            return f(*args, **kwargs)
        return decorated_function
    return decorator
```

**Using the Decorator:**
```python
@app.route('/admin')
@login_required
@permission_required(Permission.ADMINISTER)
def for_admins_only():
    return "For administrators!"
```

## **6. Extending the User Model**

Extend the `User` model to include methods that check permissions and roles.

**Check Permissions:**
```python
class User(UserMixin, db.Model):
    # existing fields...
    def can(self, permissions):
        return self.role is not None and \
            (self.role.permissions & permissions) == permissions

    def is_administrator(self):
        return self.can(Permission.ADMINISTER)
```

## **Summary**

- **User Roles:** Define and manage different user roles in the application.
- **Permissions:** Use bitmasks to manage permissions efficiently.
- **Role Assignment:** Automatically assign roles to new users.
- **RBAC:** Implement role-based access control using decorators.
- **Model Extensions:** Extend user and role models to handle permissions and roles.

By implementing these concepts, you can create a secure and well-structured application with clear role-based access controls, ensuring that users have appropriate permissions based on their roles.